In [8]:
import pandas as pd
import numpy as np
import langgraph
import json
import langchain
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Dict, List
import requests
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import yfinance as yf

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPEN_AI_KEY"))


In [14]:
class PortfolioState(TypedDict, total=False):
    user: dict
    indicators: Dict[str, Optional[float]]
    industries: list
    assets: list
    recommendation: str

In [44]:
class UserProfile:
    def __init__(self, name, risk_profile, return_requirements, preferred_assets, capital):
        self.name = name
        self.risk_profile = risk_profile
        self.return_requirements = return_requirements
        self.preferred_assets = preferred_assets
        self.capital = capital

    def to_dict(self):
        return {
            "name": self.name,
            "risk_profile": self.risk_profile,
            "return_requirements": self.return_requirements,
            "preferred_assets": self.preferred_assets,
            "capital": self.capital
        }


class UserNode:
    def __init__(self, user: UserProfile):
        self.user = user

    def __call__(self, state: PortfolioState) -> PortfolioState:
        state["user"] = self.user.to_dict()
        return state


In [30]:
#First node in LangGraph Economic Indicators
class EconomicIndicators:
    def __init__(self):
        # pull API key from env
        self.api_key = os.getenv("FRED_KEY")
        
        # store indicators (initialized as None)
        self.m2 = None
        self.unemployment = None
        self.gdp = None
        self.consumer_spending = None
        self.usd_strength = None

    #API call to fetch data from Fred
    def _fetch_from_fred(self, series_id):
        """Helper: fetch latest value from FRED"""
        url = (
            f"https://api.stlouisfed.org/fred/series/observations?"
            f"series_id={series_id}&api_key={self.api_key}&file_type=json"
        )
        resp = requests.get(url).json()
        if resp.get("observations"):
            return float(resp["observations"][-1]["value"])
        return None

    # Getters for each indicator
    def get_m2(self):
        if self.m2 is None:
            self.m2 = self._fetch_from_fred("M2SL")  # Money supply
        return self.m2

    def get_unemployment(self):
        if self.unemployment is None:
            self.unemployment = self._fetch_from_fred("UNRATE")
        return self.unemployment

    def get_gdp(self):
        if self.gdp is None:
            self.gdp = self._fetch_from_fred("A191RL1Q225SBEA")  # Real GDP growth
        return self.gdp

    def get_consumer_spending(self):
        if self.consumer_spending is None:
            self.consumer_spending = self._fetch_from_fred("PCE")  # Personal Consumption Expenditures
        return self.consumer_spending

    def get_usd_strength(self):
        if self.usd_strength is None:
            dxy = yf.Ticker("DX-Y.NYB")  # Dollar Index
            hist = dxy.history(period="1mo")
            self.usd_strength = hist["Close"].iloc[-1]
        return self.usd_strength

    def get_all(self):
        """Return all indicators in one dict"""
        return {
            "m2": self.get_m2(),
            "unemployment": self.get_unemployment(),
            "gdp": self.get_gdp(),
            "consumer_spending": self.get_consumer_spending(),
            "usd_strength": self.get_usd_strength()
        }
    
class EconomicIndicatorsNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        econ = EconomicIndicators()
        state["indicators"] = econ.get_all()
        return state


In [31]:
#Class to help choose industries

class IndustriesNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        indicators = state.get("indicators", {})

        prompt = f"""
        You are an investment strategist. 
        Here are the current economic indicators:
        {indicators}

        Based on these, pick 5 industries that are most likely to perform well.
        Return your answer strictly in JSON format:
        {{
            "industries": ["Industry1", "Industry2", "Industry3", "Industry 4", "Industry 5"],
            "reasoning": {{
                "Industry1": "reason",
                "Industry2": "reason",
                "Industry3": "reason"
                "Industry4": "reason"
                "Industry5": "reason"
            }}
        }}
        """
        response = llm.invoke(prompt)
        try:
            industries_data = json.loads(response.content)
            state["industries"] = industries_data["industries"]
        except Exception:
            # fallback: just store raw text if JSON parsing fails
            state["industries"] = [response.content]

        return state

In [32]:
class AssetClassesNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        industries = state.get("industries", [])
        user = state.get("user", {})
        indicators = state.get("indicators", {})

        prompt = f"""
        You are a financial strategist.
        Economic indicators: {indicators}
        Selected industries: {industries}
        User profile: {user}

        Based on these, choose 3–5 stock-related asset classes 
        (e.g., Tech Stocks, Energy Stocks, REITs, Dividend Stocks, Growth Stocks, 
        Small-Cap Stocks, Defensive Stocks).

        Return your answer strictly in JSON format:
        {{
            "asset_classes": ["Class1", "Class2", "Class3"],
            "reasoning": {{
                "Class1": "reason for choosing",
                "Class2": "reason for choosing",
                "Class3": "reason for choosing"
            }}
        }}
        """

        response = llm.invoke(prompt)

        try:
            asset_classes_data = json.loads(response.content)
            state["asset_classes"] = asset_classes_data["asset_classes"]
            state["asset_classes_reasoning"] = asset_classes_data.get("reasoning", {})
        except Exception:
            # fallback if parsing fails
            state["asset_classes"] = [response.content]

        return state

In [33]:
class CompaniesNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        asset_classes = state.get("asset_classes", [])
        industries = state.get("industries", [])
        indicators = state.get("indicators", {})
        user = state.get("user", {})

        prompt = f"""
        You are a professional equity analyst.  
        Economic indicators: {indicators}  
        Selected industries: {industries}  
        Chosen asset classes: {asset_classes}  
        User profile: {user}  

        Based on these, recommend 5–7 companies (stock tickers) 
        that best fit the economic environment, user risk profile, and asset classes.

        Return your answer strictly in JSON format:
        {{
            "companies": ["Ticker1", "Ticker2", "Ticker3", ...],
            "reasoning": {{
                "Ticker1": "reason for selecting",
                "Ticker2": "reason for selecting",
                "Ticker3": "reason for selecting"
            }}
        }}
        """

        response = llm.invoke(prompt)

        try:
            companies_data = json.loads(response.content)
            state["companies"] = companies_data["companies"]
            state["companies_reasoning"] = companies_data.get("reasoning", {})
        except Exception:
            # fallback: raw LLM output
            state["companies"] = [response.content]

        return state

In [34]:
class SentimentNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        companies = state.get("companies", [])
        industries = state.get("industries", [])
        indicators = state.get("indicators", {})
        user = state.get("user", {})

        if not companies:
            state["sentiment"] = {}
            return state

        prompt = f"""
        You are a market strategist.  
        Economic indicators: {indicators}  
        Industries: {industries}  
        User profile: {user}  
        Companies to evaluate: {companies}  

        For each company, assess the current market sentiment 
        ("positive", "neutral", or "negative") 
        based on macro conditions, industry outlook, and user risk profile.

        Return your answer strictly in JSON format:
        {{
            "sentiment": {{
                "Company1": "positive/neutral/negative",
                "Company2": "positive/neutral/negative",
                ...
            }},
            "reasoning": {{
                "Company1": "brief explanation",
                "Company2": "brief explanation"
            }}
        }}
        """

        response = llm.invoke(prompt)

        try:
            sentiment_data = json.loads(response.content)
            state["sentiment"] = sentiment_data.get("sentiment", {})
            state["sentiment_reasoning"] = sentiment_data.get("reasoning", {})
        except Exception:
            # fallback: store raw LLM output
            state["sentiment"] = {c: response.content for c in companies}

        return state


In [35]:
class TechnicalIndicatorsNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        companies = state.get("companies", [])
        technicals = {}

        for c in companies:
            try:
                # Download last 6 months of daily data
                data = yf.download(c, period="6mo", interval="1d", progress=False)
                close = data["Close"]

                # --- RSI (14-day) ---
                delta = close.diff()
                gain = delta.clip(lower=0).rolling(14).mean()
                loss = -delta.clip(upper=0).rolling(14).mean()
                rs = gain / loss
                rsi = 100 - (100 / (1 + rs))

                # --- MACD (12-26) ---
                ema12 = close.ewm(span=12, adjust=False).mean()
                ema26 = close.ewm(span=26, adjust=False).mean()
                macd = ema12 - ema26
                signal = macd.ewm(span=9, adjust=False).mean()

                # Save most recent values
                technicals[c] = {
                    "RSI": round(rsi.iloc[-1], 2),
                    "MACD": round(macd.iloc[-1], 2),
                    "Signal": round(signal.iloc[-1], 2),
                    "Trend": "bullish" if macd.iloc[-1] > signal.iloc[-1] else "bearish"
                }

            except Exception as e:
                technicals[c] = {"error": str(e)}

        state["technical"] = technicals
        return state


In [36]:
class FundamentalIndicatorsNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        companies = state.get("companies", [])
        fundamentals = {}

        for c in companies:
            try:
                ticker = yf.Ticker(c)

                # Try to extract key fundamentals
                info = ticker.info  # note: sometimes incomplete
                pe = info.get("forwardPE") or info.get("trailingPE")
                roe = info.get("returnOnEquity")  # usually reported as decimal
                eps = info.get("trailingEps")
                pb = info.get("priceToBook")
                debt_to_equity = info.get("debtToEquity")

                fundamentals[c] = {
                    "PE": round(pe, 2) if pe else None,
                    "ROE": round(roe, 4) if roe else None,
                    "EPS": round(eps, 2) if eps else None,
                    "P/B": round(pb, 2) if pb else None,
                    "Debt/Equity": round(debt_to_equity, 2) if debt_to_equity else None,
                }

            except Exception as e:
                fundamentals[c] = {"error": str(e)}

        state["fundamental"] = fundamentals
        return state


In [37]:
class PortfolioOptimizationNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        sentiment = state.get("sentiment", {})
        technical = state.get("technical", {})
        fundamental = state.get("fundamental", {})
        user = state.get("user", {})

        scores = {}

        for c in sentiment.keys():
            score = 0.0

            # --- Sentiment scoring ---
            if sentiment.get(c) == "positive":
                score += 2
            elif sentiment.get(c) == "neutral":
                score += 1
            else:  # negative
                score -= 1

            # --- Technical scoring ---
            t = technical.get(c, {})
            if "RSI" in t:
                if t["RSI"] < 30:     # oversold → bullish
                    score += 1
                elif t["RSI"] > 70:  # overbought → bearish
                    score -= 1
            if t.get("Trend") == "bullish":
                score += 1
            elif t.get("Trend") == "bearish":
                score -= 1

            # --- Fundamental scoring ---
            f = fundamental.get(c, {})
            pe = f.get("PE")
            roe = f.get("ROE")

            if pe and pe < 20:  # attractive valuation
                score += 1
            elif pe and pe > 35:  # expensive
                score -= 1

            if roe and roe > 0.15:  # strong profitability
                score += 1

            scores[c] = score

        # Normalize scores to weights
        total = sum(np.exp(v) for v in scores.values()) if scores else 1
        weights = {c: round(np.exp(v) / total, 3) for c, v in scores.items()}

        # Apply user risk adjustment
        risk = user.get("risk_profile", "moderate")
        if risk == "conservative":
            # Cap any single stock at 30%
            weights = {c: min(w, 0.3) for c, w in weights.items()}
            # Re-normalize
            s = sum(weights.values())
            weights = {c: round(w / s, 3) for c, w in weights.items()}

        state["portfolio"] = weights
        return state


In [38]:
class RecommendationNode:
    def __call__(self, state: PortfolioState) -> PortfolioState:
        portfolio = state.get("portfolio", {})
        user = state.get("user", {})
        industries = state.get("industries", [])
        asset_classes = state.get("asset_classes", [])

        if not portfolio:
            state["recommendation"] = {
                "portfolio": {},
                "note": "No portfolio could be generated. Please check earlier nodes."
            }
            return state

        # Build a summary note
        risk = user.get("risk_profile", "moderate")
        capital = user.get("capital", None)
        target_return = user.get("return_requirements", None)

        note_parts = []
        note_parts.append(f"Portfolio tailored for a **{risk}** investor.")

        if industries:
            note_parts.append(f"Focus on industries: {', '.join(industries)}.")
        if asset_classes:
            note_parts.append(f"Asset classes emphasized: {', '.join(asset_classes)}.")

        # Capital allocation
        if capital:
            allocation = {c: round(w * capital, 2) for c, w in portfolio.items()}
            note_parts.append(f"With capital of ${capital:,.0f}, allocations are: {allocation}.")
        else:
            note_parts.append("Weights are provided as percentages of total portfolio.")

        # Return target
        if target_return:
            note_parts.append(f"Designed to target ~{target_return*100:.1f}% annual return.")

        # Risk balance
        if max(portfolio.values()) > 0.5:
            note_parts.append("⚠ Note: High concentration risk — consider diversifying further.")
        else:
            note_parts.append("Portfolio shows a balanced allocation across multiple stocks.")

        # Final state update
        state["recommendation"] = {
            "portfolio": portfolio,
            "note": " ".join(note_parts)
        }

        return state

In [42]:
workflow = StateGraph(PortfolioState)

# Add class-based nodes
workflow.add_node("econ", EconomicIndicatorsNode())
workflow.add_node("industries", IndustriesNode())
workflow.add_node("asset_classes", AssetClassesNode())
workflow.add_node("companies", CompaniesNode())
workflow.add_node("sentiment", SentimentNode())
workflow.add_node("technical", TechnicalIndicatorsNode())
workflow.add_node("fundamental", FundamentalIndicatorsNode())
workflow.add_node("portfolio_opt", PortfolioOptimizationNode())
workflow.add_node("recommendation", RecommendationNode())

# Edges
workflow.add_edge("START", "econ")          # 👈 entrypoint
workflow.add_edge("econ", "industries")
workflow.add_edge("industries", "asset_classes")
workflow.add_edge("asset_classes", "companies")
workflow.add_edge("companies", "sentiment")

workflow.add_edge("sentiment", "technical")
workflow.add_edge("sentiment", "fundamental")

workflow.add_edge("technical", "portfolio_opt")
workflow.add_edge("fundamental", "portfolio_opt")

workflow.add_edge("portfolio_opt", "recommendation")

In [ ]:
workflow = StateGraph(PortfolioState)


# User comes first
user = UserProfile(
    name="Alice",
    risk_profile="moderate",
    return_requirements=0.08,
    preferred_assets=["Tech Stocks", "Dividend Stocks"],
    capital=100000
)

# Entry edge
workflow.add_edge("START", "user")

workflow.add_node("user", UserNode(user))
workflow.add_node("econ", EconomicIndicatorsNode())
workflow.add_node("industries", IndustriesNode())
workflow.add_node("asset_classes", AssetClassesNode())
workflow.add_node("companies", CompaniesNode())
workflow.add_node("sentiment", SentimentNode())
workflow.add_node("technical", TechnicalIndicatorsNode())
workflow.add_node("fundamental", FundamentalIndicatorsNode())
workflow.add_node("portfolio_opt", PortfolioOptimizationNode())
workflow.add_node("recommendation", RecommendationNode())

# Edges
workflow.add_edge("user", "econ")             # user is the entrypoint
workflow.add_edge("econ", "industries")
workflow.add_edge("industries", "asset_classes")
workflow.add_edge("asset_classes", "companies")
workflow.add_edge("companies", "sentiment")

workflow.add_edge("sentiment", "technical")
workflow.add_edge("sentiment", "fundamental")

workflow.add_edge("technical", "portfolio_opt")
workflow.add_edge("fundamental", "portfolio_opt")

workflow.add_edge("portfolio_opt", "recommendation")


In [46]:
app = workflow.compile()
initial_state = PortfolioState()   # no need to prefill user, node handles it
final_state = app.invoke(initial_state)

print(final_state["recommendation"])


ValueError: Graph must have an entrypoint: add at least one edge from START to another node